歪打误撞，非自己决策的开仓——赚一笔就跑

In [3]:
import tushare as ts

# 黄金和比特币趋势

## 比特币数据

In [31]:
ts.set_token("409115656486e3f61481103ec280339511aee50ee9026f9b008985ea")

pro = ts.pro_api()
df = pro.btc_pricevol(start_date='20180101', end_date='20180801')

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tushare\util\upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [32]:
df.head()

,date,price,volume
0,20180801,7579.056837,5.759918e+08
1,20180731,7915.267761,7.305237e+08
2,20180730,8128.549310,6.716885e+08
3,20180729,8193.888653,2.656748e+08
4,20180728,8168.527643,2.808106e+08


## 黄金数据

# 北向资金
沪股通：http://data.10jqka.com.cn/hgt/hgtb/

深股通：http://data.10jqka.com.cn/hgt/sgtb/

In [9]:
### 单位：百万元（不一定准确）
ts.set_token('409115656486e3f61481103ec280339511aee50ee9026f9b008985ea')
pro = ts.pro_api()

pro.moneyflow_hsgt(start_date='20200323', end_date='20200326')

,trade_date,ggt_ss,ggt_sz,hgt,sgt,north_money,south_money
0,20200325,4382.79,1358.31,1245.33,1258.53,2503.86,5741.10
1,20200324,2562.71,1702.66,3063.18,923.96,3987.14,4265.37
2,20200323,7679.66,1830.58,-6294.21,-1705.05,-7999.26,9510.24


In [10]:
import requests

In [11]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"
}
url = "http://data.10jqka.com.cn/hgt/sgtb/"

session = requests.get(url,headers=headers)

In [13]:
session.text()

TypeError: 'str' object is not callable

# 市场成交金额
http://quote.eastmoney.com/zs000001.html

http://quote.eastmoney.com/zs399001.html

http://quote.eastmoney.com/zs399006.html

In [ ]:
###取上证指数行情数据（amount：单位千元）
df = ts.pro_bar(ts_code='000001.SH', asset='I', start_date='20180101', end_date='20181011')

# 期货-股票联动
农产品、贵金属